In [1]:
import pandas as pd

In [2]:
TXT_FILE_NAME = "curation_test.txt"
TXT_FILE_ENCODING_TYPE = "utf-8"
SOURCE_FILE_PATH = f"source/{TXT_FILE_NAME}"
ANNOTATION_TSV_FILE_PATH = f"curation/{TXT_FILE_NAME}/CURATION_USER.tsv"

## Read a Source file

In [3]:
# source_file_list = open(SOURCE_FILE_PATH, 'r', encoding=TXT_FILE_ENCODING_TYPE).read().split('\n')

In [4]:
# source_file_list

## Read an Annotated TSV file

### Pre-processing a TSV file

In [5]:
annotation_tsv_file_list = open(ANNOTATION_TSV_FILE_PATH, 'r').read().split('\n')

In [6]:
idx = 1

col_list = ["#Text", "span", "token"]
tag_list = []
while True:
    if annotation_tsv_file_list[idx]=='':
        break
    
    row = annotation_tsv_file_list[idx]
    tag_name = row.split('|')[0].split('.')[-1]
    tag_list.append(tag_name)
    
    # row_attribute = []
    # if (len(row.split('|'))!=2) or (row.split('|')[1]!=''):
        # row_attribute = [tag_name+"_"+attribute for attribute in row.split('|')[1:]]
    row_attribute = [tag_name+"_"+attribute for attribute in row.split('|')[1:]]
    
    for attribute in row_attribute:
        col_list.append(attribute)
    
    idx+=1

In [7]:
print(f"tag_list : {tag_list}")
print(f"col_list : {col_list}")

tag_list : ['Inj_type', 'Int_Mint', 'Int_Noti', 'Int_Obs', 'Int_Px', 'Inv_HEAD', 'Inv_PER', 'Inv_PLA', 'Inv_THG', 'PVT_Sdevices', 'PVT_teaching', 'PVT_univeral', 'Risk_Beh', 'Risk_Cog', 'Risk_Env', 'Risk_Mob', 'Risk_Tx', 'Severity', 'Type']
col_list : ['#Text', 'span', 'token', 'Inj_type_TagLinker', 'Int_Mint_TagLinker', 'Int_Noti_TagLinker', 'Int_Obs_', 'Int_Px_TagLinker', 'Inv_HEAD_', 'Inv_PER_TagLinker', 'Inv_PLA_TagLinker', 'Inv_THG_TagLinker', 'PVT_Sdevices_TagLinker', 'PVT_teaching_', 'PVT_univeral_TagLinker', 'Risk_Beh_TagLinker', 'Risk_Cog_TagLinker', 'Risk_Env_TagLinker', 'Risk_Mob_TagLinker', 'Risk_Tx_TagLinker', 'Severity_TagLinker', 'Type_TagLinker']


In [8]:
annotation_list = annotation_tsv_file_list[idx+1:]
annotation_list[0] = "\t".join(col_list)+"\t"
for _ in range(annotation_list.count('')): # remove empty row
    annotation_list.remove('')

annotation_table = [row.split("\t") for row in annotation_list]

In [9]:
df = pd.DataFrame(annotation_table[1:], columns=annotation_table[0])
df.drop([''], axis=1, inplace=True)

In [10]:
token_pairs_list = []


idx = 0
while idx < len(df):
    is_token_tagless = True

    for tag in col_list[3:]:
        if (df[tag][idx]=='_' or df[tag][idx]==None):
            continue
        else:
            # if (df["token"][idx]==None or df["token"][idx].isDecimal()==True or p.match(df["token"][idx])):
            #     idx += 1
            #     continue

            token = df["token"][idx]
            if df[tag][idx][0] == '*':
                df[tag][idx] = df[tag][idx].replace('*', tag)
            detail_tag = df[tag][idx]
                
            token_pairs_list.append((token, detail_tag))
            is_token_tagless = False
    # end for loop
    
    if is_token_tagless==True and df["token"][idx]!=None:
        token_pairs_list.append((df["token"][idx], "O")) # Making O-tag
    idx += 1
# end while loop

In [11]:
token_pairs_list[-30:]

[('no', 'O'),
 ('pain', 'O'),
 ('and', 'O'),
 ('that', 'O'),
 ('he', 'O'),
 ('had', 'O'),
 ('no', 'O'),
 ('symptoms', 'O'),
 ('.', 'O'),
 ('No', 'none[56]'),
 ('trauma', 'none[56]'),
 ('observed', 'O'),
 ('.', 'O'),
 ('"', 'O'),
 ('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('found', 'O'),
 ('dead', 'O'),
 ('with', 'O'),
 ('a', 'O'),
 ("'", 'O'),
 ('thump', 'O'),
 ("'", 'O'),
 ('in', 'O'),
 ('front', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('elevator', 'O'),
 ('.', 'O')]

In [12]:
BIO_table = []

temp_tag_list = []
for idx, (token, tag) in enumerate(token_pairs_list):
    if tag == "O": # except O-tag
        pass
    elif tag[-1] != ']': # Making B-tag
        tag = "B-" + tag
    else: # Making sequential tag
        real_tag = tag.split('[')[0]
        if tag in temp_tag_list: # Making I-tag
            tag = "I-" + real_tag
        else: # Making B-tag
            temp_tag_list.append(tag)
            tag = "B-" + real_tag
    
    BIO_table.append([token, tag])

In [13]:
BIO_df = pd.DataFrame(BIO_table, columns=["token", "tag"])
BIO_df.to_csv(f"BIO-{TXT_FILE_NAME[:-4]}.csv")

In [14]:
BIO_df

,token,tag
0,"""",O
1,NIBP_systolic,O
2,(,O
3,mmHg,O
4,),O
...,...,...
4502,front,O
4503,of,O
4504,the,O
4505,elevator,O


In [15]:
BIO_df.describe()

,token,tag
count,4507,4507
unique,930,60
top,the,O
freq,312,4239
